In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [1]:
!nvidia-smi

Sat Apr  5 07:24:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
%ls -la

total 16
drwxr-xr-x 1 root root 4096 Apr  3 13:37 ./
drwxr-xr-x 1 root root 4096 Apr  5 07:17 ../
drwxr-xr-x 4 root root 4096 Apr  3 13:37 .config/
drwxr-xr-x 1 root root 4096 Apr  3 13:37 sample_data/


In [4]:
!nvcc vec-add.cu -o vec-add

cc1plus: fatal error: vec-add.cu: No such file or directory
compilation terminated.
